In [19]:
import requests

def get_companies_by_nace_code_and_city(nace_code, city, max_results=5000):
    # Initialiser variabler for paginering
    base_url = 'https://data.brreg.no/enhetsregisteret/api/enheter'
    page = 0
    all_companies = []
    page_size = 100  # API-et har ofte en maksgrense på 100 per side
    total_fetched = 0

    while total_fetched < max_results:
        # Sett opp URL med næringskode, poststed (by) og paginering
        url = f'{base_url}?naeringskode={nace_code}&forretningsadresse.poststed={city}&page={page}&size={page_size}'
        
        # Send forespørselen til API-et
        response = requests.get(url)

        # Sjekk om forespørselen var vellykket
        if response.status_code == 200:
            data = response.json()
            # Legg til resultatene fra denne siden til den totale listen
            companies = data["_embedded"]["enheter"]
            
            # Filtrer kun aksjeselskaper (organisasjonsformkode 'AS') og ekskluder selskaper under konkursbehandling
            as_companies = [
                company for company in companies 
                if company.get('organisasjonsform', {}).get('kode') == 'AS' and company.get('status') != 'Konkurs'
            ]
            
            all_companies.extend(as_companies)
            total_fetched += len(as_companies)

            # Sjekk om vi er på siste side (siste side har ingen "_links" -> "next")
            if "next" not in data["_links"]:
                break
            else:
                page += 1  # Gå til neste side
        else:
            print(f"Feil ved forespørsel: {response.status_code}")
            break

    # Returner kun maksimalt antall ønskede resultater (f.eks. 5000)
    return all_companies[:max_results]

# Eksempel på bruk med næringskode 47.24 og by Oslo
nace_code = "47.24"
city = "Oslo"
companies = get_companies_by_nace_code_and_city(nace_code, city)

# Nå er 'companies' variabelen filtrert for kun aksjeselskaper som ikke er under konkursbehandling
print(f"Fant {len(companies)} aksjeselskaper i Oslo med næringskode {nace_code}")


Fant 58 aksjeselskaper i Oslo med næringskode 47.24


In [20]:
print(companies)

[{'organisasjonsnummer': '932740265', 'navn': 'AJ SCHOUTEN AS', 'organisasjonsform': {'kode': 'AS', 'beskrivelse': 'Aksjeselskap', '_links': {'self': {'href': 'https://data.brreg.no/enhetsregisteret/api/organisasjonsformer/AS'}}}, 'postadresse': {'land': 'Norge', 'landkode': 'NO', 'postnummer': '0365', 'poststed': 'OSLO', 'adresse': ['c/o Alexandra Jansen Schouten', 'Fauchalds gate 9B'], 'kommune': 'OSLO', 'kommunenummer': '0301'}, 'registreringsdatoEnhetsregisteret': '2024-01-02', 'registrertIMvaregisteret': True, 'naeringskode1': {'kode': '47.241', 'beskrivelse': 'Butikkhandel med bakervarer og konditorvarer'}, 'antallAnsatte': 10, 'harRegistrertAntallAnsatte': True, 'forretningsadresse': {'land': 'Norge', 'landkode': 'NO', 'postnummer': '0352', 'poststed': 'OSLO', 'adresse': ['Oscars gate 19'], 'kommune': 'OSLO', 'kommunenummer': '0301'}, 'stiftelsesdato': '2023-10-26', 'institusjonellSektorkode': {'kode': '2100', 'beskrivelse': 'Private aksjeselskaper mv.'}, 'registrertIForetaksreg